## Ablation study on KS dataset for PFNN consist on short-term predictions after contraction

In [ ]:
cd ..

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from model.utilities import *
from model.koopman_base import *
# from models.fno_1d import *
import sys
sys.path.append('./models')

import numpy.random as random

font = {'size'   : 12, 'family': 'Times New Roman'}
matplotlib.rc('font', **font)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

# Main
n_train = 1000
n_test = 100

sub = 4 # spatial subsample
S = 512
s = S//sub

T_in = 500 # skip first 500 seconds of each trajectory to keep trajectory on attractor
T = 200 # seconds to extract from each trajectory in data
T_out = T_in + T
step = 1 # Seconds to learn solution operator

# Load data
predloader = MatReader('./data/KS.mat') # load the generated data
data_raw = predloader.read_field('u')
data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]
data_test = data_tensor[-n_test:,:,:]

test_a = data_test[:,T_in-1:T_out-1,:].reshape(-1, s)
test_u = data_test[:,T_in:T_out,:].reshape(-1, s)
batch_size = 100
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

/var/folders/r1/v7355v5n0y3cw6l975b_dspm0000gn/T/ipykernel_43559/1272648692.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]


In [ ]:
device = torch.device('cpu')

PFNN_consist_0_path = 'fill_PFNN_gamma2_0_model_path'
model_consist_0 = torch.load(PFNN_consist_0_path, map_location=device)

PFNN_consist_01_path = 'fill_PFNN_gamma2_01_model_path'
model_consist_01 = torch.load(PFNN_consist_01_path, map_location=device)

PFNN_consist_03_path = 'fill_PFNN_gamma2_03_model_path'
model_consist_03 = torch.load(PFNN_consist_03_path, map_location=device)

PFNN_consist_05_path = 'fill_PFNN_gamma2_05_model_path'
model_consist_05 = torch.load(PFNN_consist_05_path, map_location=device)

PFNN_consist_1_path = 'fill_PFNN_gamma2_1_model_path'
model_consist_1 = torch.load(PFNN_consist_1_path, map_location=device)

In [ ]:
def episode_l2_loss(pred, truth, n = 100):
    return torch.mean((pred[:n] - truth[:n])**2)

def episode_loss_collection(regressive_steps, loss_fn, test_u, pred_1, pred_2, pred_3, pred_4, pred_5):
      loss_dict = {}

      loss_consist_0 = loss_fn(pred_1, test_u, n=regressive_steps)
      loss_consist_01 = loss_fn(pred_2, test_u, n=regressive_steps)
      loss_consist_03 = loss_fn(pred_3, test_u, n=regressive_steps)
      loss_consist_05 = loss_fn(pred_4, test_u, n=regressive_steps)
      loss_consist_1 = loss_fn(pred_5, test_u, n=regressive_steps)
            
      loss_dict['consist_0'] = loss_consist_0.item()
      loss_dict['consist_01'] = loss_consist_01.item()
      loss_dict['consist_03'] = loss_consist_03.item()
      loss_dict['consist_05'] = loss_consist_05.item()
      loss_dict['consist_1'] = loss_consist_1.item()

      return loss_dict

In [ ]:
steps_n_list = np.array([100])
error_df_list = {}
for steps_n in steps_n_list:
      print('steps_n:', steps_n, 'started.')
      error_df = pd.DataFrame(columns=['consist_0', 'consist_01', 'consist_03', 'consist_05', 'consist_1'])
      # for init_id in range(test_samples):
      for init_id in tqdm(np.arange(n_test)):
            consist_0_long_pred = long_prediction(model_consist_0, test_a, init_id, 1, s, s, T=steps_n)
            consist_01_long_pred = long_prediction(model_consist_01, test_a, init_id, 1, s, s, T=steps_n)
            consist_03_long_pred = long_prediction(model_consist_03, test_a, init_id, 1, s, s, T=steps_n)
            consist_05_long_pred = long_prediction(model_consist_05, test_a, init_id, 1, s, s, T=steps_n)
            consist_1_long_pred = long_prediction(model_consist_1, test_a, init_id, 1, s, s, T=steps_n)

            episode_loss_dict = episode_loss_collection(steps_n, episode_l2_loss, test_u[int(init_id*T):], consist_0_long_pred, consist_01_long_pred, consist_03_long_pred, consist_05_long_pred, consist_1_long_pred)
            error_df.loc[init_id] = episode_loss_dict
      error_df_list['step_{}'.format(steps_n)] = error_df

steps_n: 100 started.


100%|██████████| 100/100 [00:10<00:00,  9.13it/s]


In [ ]:
error_mean_df = pd.DataFrame(
      columns=['consist_0', 'consist_01', 'consist_03', 'consist_05', 'consist_1'])
for key in error_df_list.keys():
      error_mean_df.loc[key] = (np.sqrt(error_df_list[key])).mean()
error_std_df = pd.DataFrame(
      columns=['consist_0', 'consist_01', 'consist_03', 'consist_05', 'consist_1'])
for key in error_df_list.keys():
      error_std_df.loc[key] = (np.sqrt(error_df_list[key])).std()

In [ ]:
error_mean_df

,consist_0,consist_01,consist_03,consist_05,consist_1
step_100,1.700724,1.706189,1.478013,1.009715,1.695502


In [ ]:
range_list = []
for i in range (n_test):
      range_list.append((test_u[T*i:T*(i+1)].max() - test_u[T*i:T*(i+1)].min()).item())
range_list = np.array(range_list)
range_list_rep = range_list[:,None].repeat(6, axis=1)
range_mean = range_list.mean()
range_max = range_list.max()
print('range_mean:', range_mean, 'range_max:', range_max)

range_mean: 6.3963081169128415 range_max: 6.994479179382324


In [ ]:
error_mean_percent_df = 100*error_mean_df/range_mean.item()
error_std_percent_df = 100*error_std_df/range_mean.item()

### NRMSE in percentage (for 100 steps prediction) ablation results for model trained on different Gamma2 on measure invariant loss

In [ ]:
error_mean_percent_df

,consist_0,consist_01,consist_03,consist_05,consist_1
step_100,26.589153,26.674595,23.107285,15.785907,26.507506


In [ ]:
error_std_percent_df

,consist_0,consist_01,consist_03,consist_05,consist_1
step_100,1.674041,1.739142,1.369389,1.766465,1.711914
